Flat ACF

In [ ]:
elif method=='flat':
    if u.Quantity(self.frequency[-1], u.microhertz) > u.Quantity(500., u.microhertz):
        window = 250.
    else:
        window = 25.

    numaxs = np.arange(window/2, np.floor(np.nanmax(self.frequency.value)) - window/2, 1.)
    metric = np.zeros(len(numaxs))
    for idx, numax in enumerate(numaxs):
        acf = self._autocorrelate(numax, method='flat', window=window)
        metric[idx] = (np.sum(np.abs(acf)) - 1 ) / len(acf)

H0 Test

In [ ]:
if method=='H0':
    h0 = 1.0 - np.exp(-self.power.value)      #Calculate probability signal is not noise
    det = self.frequency[h0 > 0.98].value       #Call it a detection if >98% likely
    for idx, numax in enumerate(numaxs):        #Cycle through a range of numax values
        width = self._get_fwhm(numax)           #Calculate how many detections fall within the fwhm
        inliers = len(det[np.abs(det - numax) < width]/10) / width
        metric[idx] = inliers

In [ ]:
def _autocorrelate(self, numax, method='hanning', window=250.):
    """An autocorrelation function (ACF) for seismic mode envelopes.
    We autocorrelate the region one FWHM of the mode envelope either side
    of the proposed numax.
    Before autocorrelating, it multiplies the section with a hanning
    window, which will increase the autocorrelation power if the region
    has a Gaussian shape, as we'd expect for seismic oscillations.

    Parameters:
    ----------
        numax : float
            The estimated position of the numax of the power spectrum. This
            is used to calculated the region autocorrelated with itself.

    Returns:
    --------
        acf : array-like
            The autocorrelation power calculated for the given numax
    """
    fs = np.median(np.diff(self.frequency.value))

    if method == 'hanning':
        fwhm = int(np.floor(self._get_fwhm(numax) / fs))    # Express the fwhm in indices
        # fwhm -= fwhm % 2                                  # Make the FWHM value even (%2 = 0 if even, 1 if odd)
        x = int(numax / fs)                                 #Find the index value of numax
        s = np.hanning(len(self.power[x-fwhm:x+fwhm]))      #Define the hanning window for the evaluated frequency space
        p_han = self.power[x-fwhm:x+fwhm].value * s         #Multiply the evaluated SNR space by the hanning window

    elif method == 'flat':
        spread = int(window/2/fs)
        x = int(numax / fs)
        p_han = self.power[x-spread:x+spread].value

    C = np.correlate(p_han, p_han, mode='full')         #Correlated the resulting SNR space with itself
    C = C[len(p_han)-1:]                                #Truncate the ACF
    return C
